# Notebook for developing preliminary models

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [3]:
# metric
from sklearn.metrics import mean_squared_log_error
def rmsle(y_true, y_pred, **kwargs):
    return np.sqrt(mean_squared_log_error(y_true, np.clip(y_pred, 0, None)))
rmsle_scorer = sklearn.metrics.make_scorer(rmsle, greater_is_better=False)

def rmsle_exp(y_true, y_pred, **kwargs):
    return np.sqrt(mean_squared_log_error(np.expm1(y_true), np.expm1(y_pred))) 
rmsle_scorer_exp = sklearn.metrics.make_scorer(rmsle_exp, greater_is_better=False)

In [4]:
from sklearn.model_selection import cross_val_score
def evaluate_CV(model, X, y, metric=rmsle_scorer, n_folds=5, random_state=None):
    return -cross_val_score(model, X, y, cv=n_folds, scoring=metric).mean()

## Data preparation for model training

### Importing data

In [5]:
# read in data Train data
train = pd.read_csv("train.csv", delimiter=",")
X_additional = pd.read_csv("additionalAttributes.csv", delimiter=",").drop("Unnamed: 0", axis=1)
X = train.drop(["formation_energy_ev_natom", "bandgap_energy_ev", "id"], axis=1)
y_fe = train.formation_energy_ev_natom
y_be = train.bandgap_energy_ev
X_full = pd.concat([X, X_additional], axis=1)

#Read in Test data
test = pd.read_csv("test.csv", delimiter=",")
X_test = test.drop(["id"], axis=1)
X_test_additional = pd.read_csv("additionalAttributesTest.csv", delimiter=",").drop("Unnamed: 0", axis=1)
X_full_Test = pd.concat([X_test, X_test_additional], axis=1)


In [6]:
X_full

,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,...,distGaO,distInAl,distInGa,distInO,elInt,qAl,qGa,qIn,qO,xEq
0,33,80.0,0.6250,0.3750,0.0000,9.9523,8.5513,9.1775,90.0026,90.0023,...,2.025,NaN,NaN,NaN,-41.086176,0.349004,0.345264,NaN,-0.231734,4.896453
1,194,80.0,0.6250,0.3750,0.0000,6.1840,6.1838,23.6287,90.0186,89.9980,...,1.775,0.025,0.025,1.775,-38.554098,0.344690,0.347084,NaN,-0.230392,4.905702
2,227,40.0,0.8125,0.1875,0.0000,9.7510,5.6595,13.9630,90.9688,91.1228,...,1.825,0.025,0.025,1.825,-33.957596,0.343019,0.339820,NaN,-0.228280,4.925995
3,167,30.0,0.7500,0.0000,0.2500,5.0036,5.0034,13.5318,89.9888,90.0119,...,0.025,2.925,0.025,0.025,-35.763321,0.337616,NaN,0.375759,-0.231435,4.898196
4,194,80.0,0.0000,0.6250,0.3750,6.6614,6.6612,24.5813,89.9960,90.0006,...,2.025,0.025,3.625,2.025,-36.645650,NaN,0.342521,0.359175,-0.232511,4.869110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,33,40.0,0.7500,0.2500,0.0000,4.9469,8.5014,9.1298,90.0038,90.0023,...,1.775,0.025,0.025,1.775,-39.262887,0.346869,0.345489,NaN,-0.231016,4.905482
2396,167,30.0,0.4167,0.5833,0.0000,4.9566,4.9562,13.4178,89.9938,90.0075,...,2.025,0.025,0.025,2.025,-35.323868,0.345140,0.342118,NaN,-0.228918,4.928134
2397,206,80.0,0.4375,0.5625,0.0000,9.2204,9.2200,9.2199,90.0047,90.0046,...,1.975,0.025,0.025,1.975,-38.566520,0.364599,0.330046,NaN,-0.230109,4.873503
2398,33,80.0,0.3125,0.1875,0.5000,10.6529,9.0954,9.7210,90.0015,89.9996,...,2.125,3.375,3.525,2.125,-39.874295,0.335601,0.329533,0.372616,-0.235314,4.842888


### Remove nan values From data

In [9]:
X_full.columns

Index(['spacegroup', 'number_of_total_atoms', 'percent_atom_al',
       'percent_atom_ga', 'percent_atom_in', 'lattice_vector_1_ang',
       'lattice_vector_2_ang', 'lattice_vector_3_ang',
       'lattice_angle_alpha_degree', 'lattice_angle_beta_degree',
       'lattice_angle_gamma_degree', 'Vatom', 'cAlGa', 'cAlIn', 'cAlO',
       'cGaAl', 'cGaIn', 'cGaO', 'cInAl', 'cInGa', 'cInO', 'distAlGa',
       'distAlIn', 'distAlO', 'distGaAl', 'distGaIn', 'distGaO', 'distInAl',
       'distInGa', 'distInO', 'elInt', 'qAl', 'qGa', 'qIn', 'qO', 'xEq'],
      dtype='object')

* If distance in Nan, it is reasonable to set them to very large value as the atoms are infinitely far from each other
* If Coordination number is Nan then it is reasonable to set it as 0, as no atoms are in vicinity of chosen atom
* If any of the charges have Nan values it is set to 0

In [10]:
fillValues = {'cAlGa':0, 
              'cAlIn':0, 
              'cAlO':0,
              'cGaAl':0, 
              'cGaIn':0, 
              'cGaO':0, 
              'cInAl':0, 
              'cInGa':0, 
              'cInO':0,
              'distAlGa':100,
              'distAlIn':100,
              'distAlO':100,
              'distGaAl':100,
              'distGaIn':100,
              'distGaO':100,
              'distInAl':100,
              'distInGa':100,
              'distInO':100,
              'qAl':0, 
              'qGa':0, 
              'qIn':0, 
              'qO':0}

In [11]:
X_full = X_full.fillna(value=fillValues)
X_full_Test = X_full_Test.fillna(value=fillValues)

### Categorize spacegroups

In [12]:
#Space groups possessing a point of inversion are termed centrosymmetric;
#ome space groups have no symmetry element that can change the handedness of an object; these are termed enantiomorphic space groups
def setSymmetry(X):
    centrosymmetric = []
    enantiomorphic = []
    groupEnantiomorph = [1]+list(np.arange(3,6,1))+list(np.arange(16,25,1))+list(np.arange(75,81,1))+list(np.arange(89,99,1))+list(np.arange(143,147,1))+list(np.arange(149,156,1))+list(np.arange(168,174,1))+list(np.arange(177,183,1))+list(np.arange(195,200,1))+list(np.arange(207,215,1))
    groupCentrosym = [2]+list(np.arange(10,16,1))+list(np.arange(47,75,1))+list(np.arange(83,89,1))+list(np.arange(123,143,1))+list(np.arange(147,149,1))+list(np.arange(162,168,1))+list(np.arange(175,177,1))+list(np.arange(191,195,1))+list(np.arange(200,207,1))+list(np.arange(221,231,1))
    for i in range(len(X)):
        if(X.spacegroup[i] in groupEnantiomorph):
            enantiomorphic+=[1]
            centrosymmetric+=[0]
        elif(X.spacegroup[i] in groupCentrosym):
            enantiomorphic+=[0]
            centrosymmetric+=[1]
        else:
            enantiomorphic+=[0]
            centrosymmetric+=[0]
    X["centroSym"] = centrosymmetric
    X["enantioMorph"] = enantiomorphic
    return X

In [13]:
# transform encoding
def encode_spacegroup(X):
    # 1-2 triclinic
    # 3-15 monoclinic
    # 16-74 orthorhombic
    # 75-142 tetragonal
    # 143-167 trigonal
    # 168-194 hexagonal
    # 195-230 cubic
    # [ 33 194 227 167 206  12] are the possible spacegroup values
    # onehot encode each separately
    return pd.get_dummies(X, columns=["spacegroup"])

In [14]:
X_full = setSymmetry(X_full)
X_full = encode_spacegroup(X_full)


X_full_Test = setSymmetry(X_full_Test)
X_full_Test = encode_spacegroup(X_full_Test)

In [15]:
X_full

,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,...,qO,xEq,centroSym,enantioMorph,spacegroup_12,spacegroup_33,spacegroup_167,spacegroup_194,spacegroup_206,spacegroup_227
0,80.0,0.6250,0.3750,0.0000,9.9523,8.5513,9.1775,90.0026,90.0023,90.0017,...,-0.231734,4.896453,0,0,0,1,0,0,0,0
1,80.0,0.6250,0.3750,0.0000,6.1840,6.1838,23.6287,90.0186,89.9980,120.0025,...,-0.230392,4.905702,1,0,0,0,0,1,0,0
2,40.0,0.8125,0.1875,0.0000,9.7510,5.6595,13.9630,90.9688,91.1228,30.5185,...,-0.228280,4.925995,1,0,0,0,0,0,0,1
3,30.0,0.7500,0.0000,0.2500,5.0036,5.0034,13.5318,89.9888,90.0119,120.0017,...,-0.231435,4.898196,1,0,0,0,1,0,0,0
4,80.0,0.0000,0.6250,0.3750,6.6614,6.6612,24.5813,89.9960,90.0006,119.9893,...,-0.232511,4.869110,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,40.0,0.7500,0.2500,0.0000,4.9469,8.5014,9.1298,90.0038,90.0023,90.0015,...,-0.231016,4.905482,0,0,0,1,0,0,0,0
2396,30.0,0.4167,0.5833,0.0000,4.9566,4.9562,13.4178,89.9938,90.0075,120.0007,...,-0.228918,4.928134,1,0,0,0,1,0,0,0
2397,80.0,0.4375,0.5625,0.0000,9.2204,9.2200,9.2199,90.0047,90.0046,89.9954,...,-0.230109,4.873503,1,0,0,0,0,0,1,0
2398,80.0,0.3125,0.1875,0.5000,10.6529,9.0954,9.7210,90.0015,89.9996,90.0004,...,-0.235314,4.842888,0,0,0,1,0,0,0,0


## Scale attributes to from 0 to 1

In [17]:
len(X_full_Test.columns)

43

In [18]:
len(X_full.columns)

43

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_full)
X_fullMinMax = scaler.transform(X_full)


scaler = MinMaxScaler()
scaler.fit(X_full_Test)
X_full_Test_MinMax = scaler.transform(X_full_Test)

### Scale using Standard scaler

In [20]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_full)

X_fullStandard = scaler.transform(X_full)

### Separate into Training and validation

In [21]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

trainSize = 0.85

############## Separate Train into Validation and Train for Formation energy ####################
#Without Scaling
#X_train_energ, X_val_energ, y_train_energ, y_val_energ = train_test_split(X_full, y_fe, train_size=trainSize, random_state=1)
#Min-Max Scaling
X_train_energ, X_val_energ, y_train_energ, y_val_energ = train_test_split(X_fullMinMax, y_fe, train_size=trainSize, random_state=1)
#Standard scaling
#X_train_energ, X_val_energ, y_train_energ, y_val_energ = train_test_split(X_fullStandard, y_fe, train_size=trainSize, random_state=1)


############## Separate Train into Validation and Train for Band Gap ####################
#Without Scaling
#X_train_gap, X_val_gap, y_train_gap, y_val_gap = train_test_split(X_full, y_be, train_size=trainSize, random_state=1)
#Min-Max Scaling
X_train_gap, X_val_gap, y_train_gap, y_val_gap = train_test_split(X_fullMinMax, y_be, train_size=trainSize, random_state=1)
#Standard scaling
#X_train_gap, X_val_gap, y_train_gap, y_val_gap = train_test_split(X_fullStandard, y_be, train_size=trainSize, random_state=1)


# Model training

## KNN

#### Formation energy

In [37]:
#################### KNN ##############################

results_df = pd.DataFrame(columns=['model',"Neigbors","Metric","weight",'trainError', 'valError','crossVal'])

for metr in ["manhattan", "minkowski"]:
    for i in range(3, 31, 2):
        for weight in ['uniform', 'distance']:
            knn = KNeighborsRegressor(n_neighbors=i, weights=weight, metric=metr).fit(X_train_energ, y_train_energ)

#            print("Neighbours " + str(i))
#            print("Metric " + metr)

            trainError = rmsle(y_train_energ, knn.predict(X_train_energ))
            valError = rmsle(y_val_energ, knn.predict(X_val_energ))
            cross_Val = evaluate_CV(knn, X_fullMinMax, y_fe)
            results_df = results_df.append({'model': 'KNN',"Neigbors":i,"Metric":metr,"weight":weight,"trainError":trainError, 'valError':valError, 'deltaErrors':abs(trainError-valError), 'crossVal':cross_Val}, ignore_index=True)


In [38]:
results_df.sort_values(by=["crossVal","valError", "deltaErrors"])

,model,Neigbors,Metric,weight,trainError,valError,crossVal,deltaErrors
2,KNN,5,manhattan,uniform,0.026725,0.032796,0.033299,0.006071
4,KNN,7,manhattan,uniform,0.028354,0.031726,0.033543,0.003373
6,KNN,9,manhattan,uniform,0.029653,0.032113,0.033791,0.002460
9,KNN,11,manhattan,distance,0.006386,0.031603,0.033821,0.025217
7,KNN,9,manhattan,distance,0.006386,0.031832,0.033871,0.025446
8,KNN,11,manhattan,uniform,0.030733,0.032010,0.033873,0.001278
5,KNN,7,manhattan,distance,0.006386,0.031857,0.033888,0.025471
11,KNN,13,manhattan,distance,0.006386,0.031406,0.033929,0.025020
3,KNN,5,manhattan,distance,0.006386,0.033049,0.033945,0.026663
13,KNN,15,manhattan,distance,0.006386,0.031317,0.034028,0.024931


In [39]:
bestKnnFe = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='manhattan').fit(X_train_energ, y_train_energ)

KNN	5	manhattan	uniform	0.026725	0.032796	0.033299	0.006071

#### Band Gap

In [40]:
#################### KNN ##############################
results_df = pd.DataFrame(columns=['model',"Neigbors","Metric","weight",'trainError', 'valError'])

for metr in ["manhattan", "minkowski"]:
    for i in range(3, 31, 2):
        for weight in ['uniform', 'distance']:
            knn = KNeighborsRegressor(n_neighbors=i, weights=weight, metric=metr).fit(X_train_gap, y_train_gap)

#            print("Neighbours " + str(i))
#            print("Metric " + metr)

            trainError = rmsle(y_train_gap, knn.predict(X_train_gap))
            valError = rmsle(y_val_gap, knn.predict(X_val_gap))
            cross_Val = evaluate_CV(knn, X_fullMinMax, y_be)
            results_df = results_df.append({'model': 'KNN',"Neigbors":i,"Metric":metr,"weight":weight,"trainError":trainError, 'valError':valError, 'deltaErrors':abs(trainError-valError), 'crossVal':cross_Val}, ignore_index=True)


In [41]:
results_df.sort_values(by=["crossVal","valError", "deltaErrors"])

,model,Neigbors,Metric,weight,trainError,valError,crossVal,deltaErrors
3,KNN,5,manhattan,distance,0.008884,0.086016,0.096120,0.077132
2,KNN,5,manhattan,uniform,0.076959,0.087256,0.097157,0.010296
7,KNN,9,manhattan,distance,0.008884,0.085056,0.097548,0.076172
5,KNN,7,manhattan,distance,0.008884,0.084873,0.097794,0.075989
9,KNN,11,manhattan,distance,0.008884,0.085306,0.098030,0.076422
11,KNN,13,manhattan,distance,0.008884,0.087785,0.099244,0.078901
1,KNN,3,manhattan,distance,0.008884,0.089373,0.099906,0.080489
0,KNN,3,manhattan,uniform,0.070379,0.091045,0.100146,0.020666
13,KNN,15,manhattan,distance,0.008884,0.089261,0.100634,0.080377
4,KNN,7,manhattan,uniform,0.083849,0.087318,0.100650,0.003469


KNN	5	manhattan	distance	0.008884	0.086016	0.096120	0.077132

In [44]:
bestKnnGap = KNeighborsRegressor(n_neighbors=5, weights='distance', metric='manhattan').fit(X_train_gap, y_train_gap)

## SVM

#### Formation energy

In [46]:

################ SVR ##################################

kernels = ['poly', 'rbf']
gammas=['scale', 'auto']
for ker in kernels:
    for gam in gammas:
        svm = SVR(kernel=ker, gamma=gam).fit(X_train_energ, y_train_energ)
        trainError = rmsle(y_train_energ, abs(svm.predict(X_train_energ)))
        valError = rmsle(y_val_energ, abs(svm.predict(X_val_energ)))
        cross_Val = evaluate_CV(svm, X_fullMinMax, y_fe)
        print("Kernel: " + ker)
        print("Gamma: "+ gam)
        print("Training error: "+str(trainError))
        print("Validation error: "+ str(valError))
        print("Cross Val: "+ str(cross_Val))

Kernel: poly
Gamma: scale
Training error: 0.04575511989007159
Validation error: 0.046548858967812815
Cross Val: 0.0468249760291898
Kernel: poly
Gamma: auto
Training error: 0.05368859105287863
Validation error: 0.053315590896120375
Cross Val: 0.054080919423228756
Kernel: rbf
Gamma: scale
Training error: 0.04655858501114167
Validation error: 0.04670380970846281
Cross Val: 0.04760095925247003
Kernel: rbf
Gamma: auto
Training error: 0.04692385157175928
Validation error: 0.0462874440934817
Cross Val: 0.04790557016826317


Kernel: rbf
Gamma: scale
Training error: 0.04645671148891898
Validation error: 0.04642970080484417

#### Band gap

In [48]:

################ SVR ##################################
from sklearn.svm import SVR

kernels = ['poly', 'rbf']
gammas=['scale', 'auto']
for ker in kernels:
    for gam in gammas:
        svm = SVR(kernel=ker, gamma=gam).fit(X_train_gap, y_train_gap)
        trainError = rmsle(y_train_gap, abs(svm.predict(X_train_gap)))
        valError = rmsle(y_val_gap, abs(svm.predict(X_val_gap)))
        cross_Val = evaluate_CV(svm, X_fullMinMax, y_be)
        print("#########################")
        print("Kernel: " + ker)
        print("Gamma: "+ gam)
        print("Training error: "+str(trainError))
        print("Validation error: "+ str(valError))
        print("Cross Val: "+ str(cross_Val))


#########################
Kernel: poly
Gamma: scale
Training error: 0.07647495018153275
Validation error: 0.0714004739716549
Cross Val: 0.08448192991361378
#########################
Kernel: poly
Gamma: auto
Training error: 0.16001985199198773
Validation error: 0.16198828270740068
Cross Val: 0.16357661238014196
#########################
Kernel: rbf
Gamma: scale
Training error: 0.08178966104416957
Validation error: 0.07406221371391183
Cross Val: 0.08694838534130755
#########################
Kernel: rbf
Gamma: auto
Training error: 0.09706518792476887
Validation error: 0.08451282750292406
Cross Val: 0.09767289649953019


Kernel: poly
Gamma: scale
Training error: 0.07584564240632702
Validation error: 0.07156753875472792

### Random Forest

#### Formation energy

In [51]:
############### Random Forest ##############################

n_estimatorss = [650,750,850,1000]
max_depths=[30,35,40,45]
min_samples_splits=[4,6,8]
seeds = [1]
results_df = pd.DataFrame(columns=['model',"seed","n_estimators","max_depth","min_samples_split",'TrainError', 'ValError', 'deltaErrors'])

for seed in seeds:
    for n_estimator in n_estimatorss:
        for max_d in max_depths:
            for min_ss in min_samples_splits:
                rf = RandomForestRegressor(criterion='mse', n_estimators=n_estimator, max_depth=max_d, min_samples_split=min_ss, random_state=seed).fit(X_train_energ, y_train_energ)
                trainError = rmsle(y_train_energ, (rf.predict(X_train_energ)))
                valError = rmsle(y_val_energ, (rf.predict(X_val_energ)))

                cross_Val = evaluate_CV(rf, X_fullMinMax, y_fe)
                results_df = results_df.append({'model': 'RF',"seed":seed,"n_estimators":n_estimator,"max_depth":max_d,"min_samples_split":min_ss,"TrainError":trainError, 'ValError':valError, 'deltaErrors':abs(trainError-valError),'crossVal':cross_Val}, ignore_index=True)


KeyboardInterrupt: 

In [ ]:
results_df.sort_values(by=["crossVal","ValError", "deltaErrors"])

39	RF	1	1000	35	4	0.013313	0.030207	0.016894	0.031341

In [ ]:
bestRfFe = RandomForestRegressor(criterion='mse', n_estimators=1000, max_depth=35, min_samples_split=4, random_state=1).fit(X_train_energ, y_train_energ)

#### Band Gap

In [ ]:
############### Random Forest ##############################

n_estimatorss = [650,750,850,1000]
max_depths=[30,35,40,45,50]
min_samples_splits=[4,6,8]
seeds = [1]
results_df = pd.DataFrame(columns=['model',"seed","n_estimators","max_depth","min_samples_split",'TrainError', 'ValError', 'deltaErrors'])

for seed in seeds:
    for n_estimator in n_estimatorss:
        for max_d in max_depths:
            for min_ss in min_samples_splits:
                rf = RandomForestRegressor(criterion='mse', n_estimators=n_estimator, max_depth=max_d, min_samples_split=min_ss, random_state=seed).fit(X_train_gap, y_train_gap)
                trainError = rmsle(y_train_gap, (rf.predict(X_train_gap)))
                valError = rmsle(y_val_gap, (rf.predict(X_val_gap)))
                cross_Val = evaluate_CV(rf, X_fullMinMax, y_be)
                results_df = results_df.append({'model': 'RF',"seed":seed,"n_estimators":n_estimator,"max_depth":max_d,"min_samples_split":min_ss,"TrainError":trainError, 'ValError':valError, 'deltaErrors':abs(trainError-valError),'crossVal':cross_Val}, ignore_index=True)


In [33]:
results_df.sort_values(by=["crossVal","ValError", "deltaErrors"])

,model,seed,n_estimators,max_depth,min_samples_split,TrainError,ValError,deltaErrors,crossVal
31,RF,1,850,30,6,0.041993,0.080487,0.038494,0.089325
34,RF,1,850,35,6,0.041993,0.080487,0.038494,0.089325
37,RF,1,850,40,6,0.041993,0.080487,0.038494,0.089325
40,RF,1,850,45,6,0.041993,0.080487,0.038494,0.089325
43,RF,1,850,50,6,0.041993,0.080487,0.038494,0.089325
33,RF,1,850,35,4,0.038443,0.080184,0.041741,0.089339
36,RF,1,850,40,4,0.038443,0.080184,0.041741,0.089339
39,RF,1,850,45,4,0.038443,0.080184,0.041741,0.089339
42,RF,1,850,50,4,0.038443,0.080184,0.041741,0.089339
30,RF,1,850,30,4,0.038444,0.080191,0.041747,0.089339


31	RF	1	850	30	6	0.041993	0.080487	0.038494	0.089325

In [ ]:
bestRfGap = RandomForestRegressor(criterion='mse', n_estimators=850, max_depth=30, min_samples_split=6, random_state=1).fit(X_train_gap, y_train_gap)

### Neutral Net (Multi-Layer Perceptron regressor)

#### Formation energy

In [35]:
############### ANN ##############################
hidden_layer_sizes = [(100,),(50,50), (50,100,50)]
activations = ["identity", "logistic", "tanh", "relu"]
solvers=['lbfgs','adam']
learning_rates = ["constant", "invscaling", "adaptive"]
seeds = [1]
results_df = pd.DataFrame(columns=['activation',"hidden_layer_size","solver","learning_rate",'TrainError', 'ValError', 'deltaErrors'])

for activation in activations:
    for solver in solvers:
        for learning_rate in learning_rates:
            for hidden_layer_size in hidden_layer_sizes:
                mlp = MLPRegressor(activation=activation, solver=solver, learning_rate=learning_rate, hidden_layer_sizes=hidden_layer_size).fit(X_train_energ, y_train_energ)
                trainError = rmsle(y_train_energ, abs(mlp.predict(X_train_energ)))
                valError = rmsle(y_val_energ, abs(mlp.predict(X_val_energ)))
                cross_Val = evaluate_CV(mlp, X_fullMinMax, y_fe)
                results_df = results_df.append({'activation':activation,"hidden_layer_size":hidden_layer_size,"solver":solver,"learning_rate":learning_rate,"TrainError":trainError, 'ValError':valError, 'deltaErrors':abs(trainError-valError),'crossVal':cross_Val}, ignore_index=True)


In [36]:
results_df.sort_values(by=["crossVal","ValError", "deltaErrors"])

,activation,hidden_layer_size,solver,learning_rate,TrainError,ValError,deltaErrors,crossVal
58,relu,"(50, 50)",lbfgs,invscaling,0.030703,0.030816,0.000114,0.031325
54,relu,"(100,)",lbfgs,constant,0.026771,0.030554,0.003783,0.031591
60,relu,"(100,)",lbfgs,adaptive,0.027959,0.030064,0.002105,0.031689
41,tanh,"(50, 100, 50)",lbfgs,invscaling,0.030880,0.030808,0.000072,0.031788
57,relu,"(100,)",lbfgs,invscaling,0.027824,0.030575,0.002751,0.031805
...,...,...,...,...,...,...,...,...
35,logistic,"(50, 100, 50)",adam,adaptive,0.052076,0.051262,0.000814,0.065256
29,logistic,"(50, 100, 50)",adam,constant,0.053991,0.052534,0.001457,0.066034
23,logistic,"(50, 100, 50)",lbfgs,invscaling,0.086962,0.085022,0.001940,0.086544
26,logistic,"(50, 100, 50)",lbfgs,adaptive,0.086831,0.084814,0.002017,0.086601


58	relu	(50, 50)	lbfgs	invscaling	0.030703	0.030816	0.000114	0.031325

#### Band gap

In [37]:
############### ANN ##############################
hidden_layer_sizes = [(100,),(50,50), (50,100,50),(50,100,100,50)]
activations = ["identity", "logistic", "tanh", "relu"]
solvers=['lbfgs','adam']
learning_rates = ["constant", "invscaling", "adaptive"]
seeds = [1]
results_df = pd.DataFrame(columns=['activation',"hidden_layer_size","solver","learning_rate",'TrainError', 'ValError', 'deltaErrors'])

for activation in activations:
    for solver in solvers:
        for learning_rate in learning_rates:
            for hidden_layer_size in hidden_layer_sizes:
                mlp = MLPRegressor(activation=activation, solver=solver, learning_rate=learning_rate, hidden_layer_sizes=hidden_layer_size).fit(X_train_gap, y_train_gap)
                trainError = rmsle(y_train_gap, abs(mlp.predict(X_train_gap)))
                valError = rmsle(y_val_gap, abs(mlp.predict(X_val_gap)))
                cross_Val = evaluate_CV(mlp, X_fullMinMax, y_be)
                results_df = results_df.append({'activation':activation,"hidden_layer_size":hidden_layer_size,"solver":solver,"learning_rate":learning_rate,"TrainError":trainError, 'ValError':valError, 'deltaErrors':abs(trainError-valError),'crossVal':cross_Val}, ignore_index=True)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

In [38]:
results_df.sort_values(by=["crossVal","ValError", "deltaErrors"])

,activation,hidden_layer_size,solver,learning_rate,TrainError,ValError,deltaErrors,crossVal
41,tanh,"(50, 100, 50)",lbfgs,invscaling,0.076627,0.073540,0.003087,0.082592
43,tanh,"(50, 50)",lbfgs,adaptive,0.076890,0.075026,0.001864,0.082881
38,tanh,"(50, 100, 50)",lbfgs,constant,0.078058,0.071912,0.006146,0.082995
44,tanh,"(50, 100, 50)",lbfgs,adaptive,0.077632,0.072404,0.005228,0.083001
40,tanh,"(50, 50)",lbfgs,invscaling,0.077870,0.073002,0.004868,0.083208
...,...,...,...,...,...,...,...,...
9,identity,"(100,)",adam,constant,0.113267,0.091627,0.021640,0.113185
10,identity,"(50, 50)",adam,constant,0.110986,0.090602,0.020384,0.113203
13,identity,"(50, 50)",adam,invscaling,0.114022,0.093414,0.020608,0.113265
12,identity,"(100,)",adam,invscaling,0.113628,0.092374,0.021254,0.114066


41	tanh	(50, 100, 50)	lbfgs	invscaling	0.076627	0.073540	0.003087	0.082592

## Best models

Using the best trained models we form a weighted average ensemble

### Formation Energy

In [22]:
resultsFe = pd.DataFrame()

In [23]:
bestKnnFe = KNeighborsRegressor(n_neighbors=5, weights='uniform', metric='manhattan').fit(X_train_energ, y_train_energ)
cross_KNN_Fe = evaluate_CV(bestKnnFe, X_fullMinMax, y_fe)
valErrorKNN = rmsle(y_val_energ, (bestKnnFe.predict(X_val_energ)))
resultKNNFe = bestKnnFe.predict(X_full_Test_MinMax)
resultsFe["KNN"] = resultKNNFe
print(valErrorKNN)

0.03279555483040591


In [24]:
#Leaving this model out as this is noticeably worse model
#bestSvmFe = SVR(kernel="rbf", gamma="scale").fit(X_train_energ, y_train_energ)
#cross_SVM_Fe = evaluate_CV(bestSvmFe, X_fullMinMax, y_fe)
#valErrorSVM = rmsle(y_val_energ, (bestSvmFe.predict(X_val_energ)))
#resultSVMFe = bestSvmFe.predict(X_val_energ)
#resultsFe["SVM"] = resultSVMFe
#print(valErrorSVM)

In [25]:
bestRfFe = RandomForestRegressor(criterion='mse', n_estimators=1000, max_depth=35, min_samples_split=4, random_state=1).fit(X_train_energ, y_train_energ)
cross_RF_Fe = evaluate_CV(bestRfFe, X_fullMinMax, y_fe)
valErrorRF = rmsle(y_val_energ, (bestRfFe.predict(X_val_energ)))
resultRFFe = bestRfFe.predict(X_full_Test_MinMax)
resultsFe["RF"] = resultRFFe
print(valErrorRF)

0.030167500161596885


In [26]:
bestMlpFe = MLPRegressor(activation="relu", solver="lbfgs", learning_rate="invscaling", hidden_layer_sizes=(50, 50)).fit(X_train_energ, y_train_energ)
cross_Mlp_Fe = evaluate_CV(bestMlpFe, X_fullMinMax, y_fe)
valErrorMLP = rmsle(y_val_energ, (bestMlpFe.predict(X_val_energ)))
resultMlpFe = bestMlpFe.predict(X_full_Test_MinMax)
resultsFe["MLP"] = resultMlpFe
print(valErrorMLP)

0.03154900116513413


#### Use Cross-Ref error as weighting factor and evaluate Validation

In [27]:
summaryError = cross_KNN_Fe+cross_RF_Fe+cross_Mlp_Fe
resultsFe["Weighted"] = cross_KNN_Fe/summaryError*resultsFe["KNN"] + cross_RF_Fe/summaryError*resultsFe["RF"] + cross_Mlp_Fe/summaryError*resultsFe["MLP"]

#errorWeighted = rmsle(y_val_energ, resultsFe["Weighted"])

In [28]:
#print(errorWeighted)

### Band Gap

In [29]:
resultsGap = pd.DataFrame()

In [30]:
"""
bestKnnGap = KNeighborsRegressor(n_neighbors=5, weights='distance', metric='manhattan').fit(X_train_gap, y_train_gap)
cross_KNN_Gap = evaluate_CV(bestKnnGap, X_fullMinMax, y_be)

valErrorKNN = rmsle(y_val_gap, (bestKnnGap.predict(X_val_gap)))
resultKNNGap = bestKnnGap.predict(X_full_Test_MinMax)
resultsGap["KNN"] = resultKNNGap
print(valErrorKNN)
"""

'\nbestKnnGap = KNeighborsRegressor(n_neighbors=5, weights=\'distance\', metric=\'manhattan\').fit(X_train_gap, y_train_gap)\ncross_KNN_Gap = evaluate_CV(bestKnnGap, X_fullMinMax, y_be)\n\nvalErrorKNN = rmsle(y_val_gap, (bestKnnGap.predict(X_val_gap)))\nresultKNNGap = bestKnnGap.predict(X_full_Test_MinMax)\nresultsGap["KNN"] = resultKNNGap\nprint(valErrorKNN)\n'

In [31]:
bestSvmGap = SVR(kernel="poly", gamma="scale").fit(X_train_gap, y_train_gap)
cross_SVM_Gap = evaluate_CV(bestSvmGap, X_fullMinMax, y_be)

valErrorSVM = rmsle(y_val_gap, (bestSvmGap.predict(X_val_gap)))
resultSVMGap = bestSvmGap.predict(X_full_Test_MinMax)
resultsGap["SVM"] = resultSVMGap
print(valErrorSVM)

0.0714004739716549


In [32]:
"""
bestRfGap = RandomForestRegressor(criterion='mse', n_estimators=850, max_depth=30, min_samples_split=6, random_state=1).fit(X_train_gap, y_train_gap)
cross_RF_Gap = evaluate_CV(bestRfGap, X_fullMinMax, y_be)

valErrorRF = rmsle(y_val_gap, (bestRfGap.predict(X_val_gap)))
resultRFGap = bestRfGap.predict(X_full_Test_MinMax)
resultsGap["RF"] = resultRFGap
print(valErrorRF)
"""

'\nbestRfGap = RandomForestRegressor(criterion=\'mse\', n_estimators=850, max_depth=30, min_samples_split=6, random_state=1).fit(X_train_gap, y_train_gap)\ncross_RF_Gap = evaluate_CV(bestRfGap, X_fullMinMax, y_be)\n\nvalErrorRF = rmsle(y_val_gap, (bestRfGap.predict(X_val_gap)))\nresultRFGap = bestRfGap.predict(X_full_Test_MinMax)\nresultsGap["RF"] = resultRFGap\nprint(valErrorRF)\n'

In [33]:
len(X_full_Test_MinMax)

600

In [34]:
bestMlpGap = MLPRegressor(activation="tanh", solver="lbfgs", learning_rate="invscaling", hidden_layer_sizes=(50, 100, 50)).fit(X_train_gap, y_train_gap)
cross_Mlp_Gap = evaluate_CV(bestMlpFe, X_fullMinMax, y_be)

valErrorMlp = rmsle(y_val_gap, (bestMlpGap.predict(X_val_gap)))
resultMlpGap = bestMlpGap.predict(X_full_Test_MinMax)
resultsGap["MLP"] = resultMlpGap
print(valErrorMlp)

0.07282709000517473


#### Use Cross-Ref error as weighting factor and evaluate Validation

In [35]:
#summaryError = cross_SVM_Gap+cross_RF_Gap+cross_Mlp_Gap
#resultsGap["Weighted"] = cross_SVM_Gap/summaryError*resultsGap["SVM"] + cross_RF_Gap/summaryError*resultsGap["RF"] + cross_Mlp_Gap/summaryError*resultsGap["MLP"]

summaryError = cross_SVM_Gap+cross_Mlp_Gap
resultsGap["Weighted"] = cross_SVM_Gap/summaryError*resultsGap["SVM"] + cross_Mlp_Gap/summaryError*resultsGap["MLP"]


#errorWeighted = rmsle(y_val_gap, resultsGap["Weighted"])

In [36]:
#errorWeighted

In [37]:
# submission file
def save_results(y_fe_pred, y_be_pred, name):
    results = pd.DataFrame({"id": test.id, "formation_energy_ev_natom": y_fe_pred, "bandgap_energy_ev": y_be_pred})
    results.to_csv(name + ".csv",index=False)

In [39]:
save_results(resultsFe["RF"] ,resultsGap["SVM"], "output2")